In [1]:
"""
General Data Science Packages
"""
import numpy as np
import pandas as pd
import geopandas as gpd
# import fiona
# import shapely
# from shapely.geometry import shape

"""
Data Managment Packages
"""
# import time
# import os
import ast

"""
Geocoding Packages
"""
# import geopy as gp
# from geopy.geocoders import Nominatim
# from geopy.extra.rate_limiter import RateLimiter
# from functools import partial

"""
Distance Calculations
"""
# from geopy.distance import geodesic
# from geopy.distance import great_circleQ

"""
Check Python Version
"""
!python --version

Python 3.8.3


# 2. Generate Gazetteer

## 2.0 Prepare the survey multilocations data

In [2]:
# csv of multilocations as tuples
mls_ = pd.read_csv('data_gen/survey_multilocations/survey_multilocations_tuples.csv')

# csv of multilocations as delimited string (alternative)
# mls_ = pd.read_csv('data_gen/survey_multilocations/survey_multilocations_string.csv')

print('\nRows, Columns: {}'.format(mls_.shape))
mls_.head(3)


Rows, Columns: (1644, 45)


,ID,109,127,134,142,152,201,202,206,211,...,100b,101,302,308,229w,229x,229y,229z,231,238
0,276785,NaN,NaN,"(('Kawango', 'Kisumu', 'Kenya'), ('Kawango', '...",NaN,NaN,"(('Kawango', 'Kisumu', 'Kenya'),)","(('Manyatta', 'Kisumu', 'Kenya'), ('Kawangware...",NaN,NaN,...,"(('-1', 'Kawango', 'Kenya'),)","(('Kisumu', 'Kisumu', 'Kenya'),)","(('-3', 'Nairobi', 'Kenya'),)","(('-3', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)","(('Kawangware', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)","(('Kawangware', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)"
1,276788,NaN,"(('N/A', 'N/A', 'N/A'),)","(('N/A', 'N/A', 'N/A'),)",NaN,NaN,NaN,NaN,NaN,NaN,...,"(('-1', 'Kokal', 'Kenya'),)","(('Oyugis', 'Oyugis', 'Kenya'),)","(('-1', '-1', '-1'),)","(('-1', '-1', '-1'),)","(('-1', '-1', '-1'),)","(('-1', '-1', '-1'),)","(('-1', '-1', '-1'),)","(('-1', '-1', '-1'),)","(('-1', '-1', '-1'),)","(('-1', '-1', '-1'),)"
2,276802,NaN,NaN,"(('Awendo', 'Migori', 'Kenya'),)",NaN,NaN,"(('Machakos', 'Machakos', 'Kenya'), ('Utawala'...",NaN,NaN,NaN,...,"(('-1', 'Oyugjs', 'Kenya'),)","(('Oyugis', 'Oyugis', 'Kenya'),)","(('Utawala', 'Nairobi', 'Kenya'),)","(('Nairobi Cbd', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)","(('Roysambu', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)","(('Roysambu', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)"


In [3]:
# run to reset the working dataframe to original dataframe
mls = mls_.copy()

### 2.0.a. Prepare the index of the input data

##### Set the index to 'ID' column

In [4]:
# check for any potential row-wise errors
print('\nDuplicate rows?:')
print(mls.duplicated().value_counts())

print('\nUnique ID column?:')
print(mls['ID'].is_unique)

# duplicate the respondent ID column and set it as the index
mls = mls.set_index('ID')

# check if the uniqueness was maintained
print('\nUnique ID index?:')
print(mls.index.is_unique)

mls.head(3)


Duplicate rows?:
False    1644
dtype: int64

Unique ID column?:
True

Unique ID index?:
True


,109,127,134,142,152,201,202,206,211,214b,...,100b,101,302,308,229w,229x,229y,229z,231,238
ID,,,,,,,,,,,,,,,,,,,,,
276785,NaN,NaN,"(('Kawango', 'Kisumu', 'Kenya'), ('Kawango', '...",NaN,NaN,"(('Kawango', 'Kisumu', 'Kenya'),)","(('Manyatta', 'Kisumu', 'Kenya'), ('Kawangware...",NaN,NaN,NaN,...,"(('-1', 'Kawango', 'Kenya'),)","(('Kisumu', 'Kisumu', 'Kenya'),)","(('-3', 'Nairobi', 'Kenya'),)","(('-3', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)","(('Kawangware', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)","(('Kawangware', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)"
276788,NaN,"(('N/A', 'N/A', 'N/A'),)","(('N/A', 'N/A', 'N/A'),)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"(('-1', 'Kokal', 'Kenya'),)","(('Oyugis', 'Oyugis', 'Kenya'),)","(('-1', '-1', '-1'),)","(('-1', '-1', '-1'),)","(('-1', '-1', '-1'),)","(('-1', '-1', '-1'),)","(('-1', '-1', '-1'),)","(('-1', '-1', '-1'),)","(('-1', '-1', '-1'),)","(('-1', '-1', '-1'),)"
276802,NaN,NaN,"(('Awendo', 'Migori', 'Kenya'),)",NaN,NaN,"(('Machakos', 'Machakos', 'Kenya'), ('Utawala'...",NaN,NaN,NaN,NaN,...,"(('-1', 'Oyugjs', 'Kenya'),)","(('Oyugis', 'Oyugis', 'Kenya'),)","(('Utawala', 'Nairobi', 'Kenya'),)","(('Nairobi Cbd', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)","(('Roysambu', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)","(('Roysambu', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)"


### 2.0.b. Prepare the columns of the input data

##### Columns and Column Positions within the Data

In [5]:
# print columns and their position in the data
for i, col in enumerate(list(mls.columns)):
    print(str(i) + '\t' + str(col))

0	109
1	127
2	134
3	142
4	152
5	201
6	202
7	206
8	211
9	214b
10	215
11	219
12	225
13	311
14	321
15	340
16	341
17	342
18	343
19	344
20	345
21	406
22	516
23	701
24	703
25	705
26	707
27	712
28	713
29	715
30	717
31	9J
32	9A
33	9N
34	100b
35	101
36	302
37	308
38	229w
39	229x
40	229y
41	229z
42	231
43	238


### 2.0.c. Prepare the elements of the input data

##### Function to evaluate the strings of the CSV as tuple literals

In [6]:
# function to map over the string elements of the dataframe
def str_eval(e):
    """
    A function to evaluate string element in a dataframe literally, such as the multilocation tuples.
    
    RETURNS: The Python literal. 
    """
    # 01 | Ensure the element is read by ast.literal_eval() as a string to avoid errors.
    e = str(e)
    
    # 02 | Filter out any null values
    if (e != 'nan'):
        
        # 03 | Evaluate the string literally
        try:
            ml = ast.literal_eval(e)
        
        # If unable to read the string, print it so it can be traced back to address issues
        except:
            print(e)
            ml = None
    
    # Return null values as null values
    else:
        ml = None
        
    return ml

##### Function to split the delimited strings in the CSV file

In [7]:
# function to split
def delimit(string):
    """
    A function that splits a string by its delimiters
    
    RETURNS: Separated strings in an array
    """
    # 01 | split the string into discrete entries using delimiter '~'
    subs = str(string).split('~')
    
    # 02 | split the entry string into discrete component using delimiter '^'
    a = [str(item).split('^') for item in subs]
    
    return a

# function to filter and split
def str_split(e):
    """
    A function that splits a string by its delimiters
    
    RETURNS: Separated strings in an array
    """
    # 01 | Ensure the element is read by ast.literal_eval() as a string to avoid errors.
    e = str(e)
    
    # 02 | Filter out any null values
    if (e != 'nan'):
        a = delimit(e)
    
    else:
        a = None
    
    return a

def a_to_tuple(a):
    """
    A function that converts a two dimensional array into nested tuples
    
    RETURNS: Multilocation tuples in a tuple.
    """
    # 01 | filter out any empty arrays or NoneTypes from tuple conversion
    # * important to evaluate the tuples literally from the csv without errors
    if a is not None:
        if (len(a) > 0):
            try:
                # 02 | tuple all the items in the array, then tuple the array
                t = tuple([tuple(ml) for ml in a])

                # 03 | ensure the element of tuples have '' before export
                # * important to evaluate the tuples literally from the csv without errors
                t = str(t) 
            except:
                print(a)
                t = None

        else:
            # return empty arrays as null values
            t = None
    else:
        # return nonetypes as null values
        t = None
        
    return t

##### Map the applicable function over the imported CSV

In [8]:
# Map the function
# evaluate tuple literals
mls = mls.applymap(lambda x: str_eval(str(x)))

# split delimited string (alternative)
# mls = mls.applymap(lambda x: str_split(str(x)))

# Preview
print('\nRows, Columns: {}'.format(mls_.shape))
display(mls.head(3))


Rows, Columns: (1644, 45)


,109,127,134,142,152,201,202,206,211,214b,...,100b,101,302,308,229w,229x,229y,229z,231,238
ID,,,,,,,,,,,,,,,,,,,,,
276785,None,None,"((Kawango, Kisumu, Kenya), (Kawango, Kisumu, K...",None,None,"((Kawango, Kisumu, Kenya),)","((Manyatta, Kisumu, Kenya), (Kawangware, Nairo...",None,None,None,...,"((-1, Kawango, Kenya),)","((Kisumu, Kisumu, Kenya),)","((-3, Nairobi, Kenya),)","((-3, Nairobi, Kenya),)","((-1, Nairobi, Kenya),)","((Kawangware, Nairobi, Kenya),)","((-1, Nairobi, Kenya),)","((-1, Nairobi, Kenya),)","((Kawangware, Nairobi, Kenya),)","((-1, Nairobi, Kenya),)"
276788,None,"((N/A, N/A, N/A),)","((N/A, N/A, N/A),)",None,None,None,None,None,None,None,...,"((-1, Kokal, Kenya),)","((Oyugis, Oyugis, Kenya),)","((-1, -1, -1),)","((-1, -1, -1),)","((-1, -1, -1),)","((-1, -1, -1),)","((-1, -1, -1),)","((-1, -1, -1),)","((-1, -1, -1),)","((-1, -1, -1),)"
276802,None,None,"((Awendo, Migori, Kenya),)",None,None,"((Machakos, Machakos, Kenya), (Utawala, Nairob...",None,None,None,None,...,"((-1, Oyugjs, Kenya),)","((Oyugis, Oyugis, Kenya),)","((Utawala, Nairobi, Kenya),)","((Nairobi Cbd, Nairobi, Kenya),)","((-1, Nairobi, Kenya),)","((Roysambu, Nairobi, Kenya),)","((-1, Nairobi, Kenya),)","((-1, Nairobi, Kenya),)","((Roysambu, Nairobi, Kenya),)","((-1, Nairobi, Kenya),)"


In [9]:
# Diagnostics
# all columns should now be object datatypes
print(mls.dtypes)

109     object
127     object
134     object
142     object
152     object
201     object
202     object
206     object
211     object
214b    object
215     object
219     object
225     object
311     object
321     object
340     object
341     object
342     object
343     object
344     object
345     object
406     object
516     object
701     object
703     object
705     object
707     object
712     object
713     object
715     object
717     object
9J      object
9A      object
9N      object
100b    object
101     object
302     object
308     object
229w    object
229x    object
229y    object
229z    object
231     object
238     object
dtype: object


## 2.1 Construct a Gazetteer of Multilocations for Export and Cleaning

The multilocations in the tabular pandas dataframe structure is effective and preserving the relationship between respondents (rows) and questions (columns) similar to the raw dataset, throughout our operations up until this point, and will continue to be a frequently referenced dataset.

For the upcoming steps of __data cleaning__ , __querying__ , and __geocoding__ , however, we will want to condense the data to run through these processes efficiently possible. We will want to determine the unique locations to avoid having to clean or query duplicate values in the dataset.

### 2.1.a. Restructure the dataframe into a flat list

While determining the unique multilocations, we no longer need to preserve the hierarchy and identity of each multilocation through the nested tuples. We can create a 1-dimenional, flat list of the multilocations before culling the duplicates.

In [10]:
# flat list to store all the multilocations tuples
mls_list = []

print('\n# mls \t - \t survey column \n')

# for every column
for c in list(mls.columns):
    
    # 01 | convert the column into a flat list of multilocations
    c_mls_list = [item for sublist in mls[c] if sublist is not None for item in sublist]
    
    # Add a row for the column to the diagnostic
    print("{} \t - \t column: '{}'".format(len(c_mls_list), c))
    
    # 02 | append the flat list for the column to the flat list for all the columns
    mls_list = mls_list + c_mls_list

# total summary
print('------')    
print('{} \t - \t Total valid multilocation responses in all columns'.format(len(mls_list)))


# mls 	 - 	 survey column 

81 	 - 	 column: '109'
1068 	 - 	 column: '127'
2088 	 - 	 column: '134'
1832 	 - 	 column: '142'
278 	 - 	 column: '152'
2558 	 - 	 column: '201'
927 	 - 	 column: '202'
287 	 - 	 column: '206'
90 	 - 	 column: '211'
28 	 - 	 column: '214b'
40 	 - 	 column: '215'
1686 	 - 	 column: '219'
1223 	 - 	 column: '225'
1092 	 - 	 column: '311'
422 	 - 	 column: '321'
688 	 - 	 column: '340'
387 	 - 	 column: '341'
469 	 - 	 column: '342'
293 	 - 	 column: '343'
903 	 - 	 column: '344'
99 	 - 	 column: '345'
1702 	 - 	 column: '406'
437 	 - 	 column: '516'
647 	 - 	 column: '701'
403 	 - 	 column: '703'
452 	 - 	 column: '705'
83 	 - 	 column: '707'
334 	 - 	 column: '712'
860 	 - 	 column: '713'
1173 	 - 	 column: '715'
731 	 - 	 column: '717'
1644 	 - 	 column: '9J'
1644 	 - 	 column: '9A'
1644 	 - 	 column: '9N'
1644 	 - 	 column: '100b'
1644 	 - 	 column: '101'
1644 	 - 	 column: '302'
1644 	 - 	 column: '308'
1644 	 - 	 column: '229w'
1644 	 - 	 column: '229x

### 2.1.b. Cull duplicates by converting flat list to a set

We can use __set()__ from base python to convert the mulitlocations in the list sequence into a list of _distinct_ elements. 

As can be seen when comparing the lengths of the multilocations lists and multilocations set, the number of unique or distinct elements is substantially lower.

In [11]:
# convert to a set
mls_set = set(mls_list)

# compare lengths
print( 'Length of flat list: {}'.format(len(mls_list)) )
print( 'Length of set:  {}'.format(len(mls_set))  )

Length of list: 45353
Length of set:  9079


### 2.1.c. Create the initial columns of the gazetteer by converting the set to a dataframe

After creating the set, we can create the first columns of the gazetteer before exporting for the manual cleaning of the data.

In [12]:
# 01 | Convert the list of tuples into the gazetteer dataframe
G = pd.DataFrame(mls_set, columns=['L1_orig', 'L2_orig', 'L3_orig'])

# 02 | Add a column with the tuples
G['tuple_orig'] = mls_set

# 03 | Sort alphabetically and reset index
G = G.sort_values(by='tuple_orig') \
     .reset_index() \
     .drop(columns=['index'])

# Diagnostic
print('\nUnique tuples?:')
print(G['tuple_orig'].is_unique)
print('\nUnique index?:')
print(G.index.is_unique)

# Preview
G


Unique tuples?:
True

Unique index?:
True


,L1_orig,L2_orig,L3_orig,tuple_orig
0,,,,"(, , )"
1,Exhibition,Accra,Ghana,"( Exhibition , Accra, Ghana)"
2,Kawangware,Nairobi,Kenya,"( Kawangware , Nairobi, Kenya)"
3,Kayole,Nairobi,Kenya,"( Kayole, Nairobi, Kenya)"
4,Masimba Junction,Nairobi,Kenya,"( Masimba Junction , Nairobi, Kenya)"
...,...,...,...,...
9074,uvira,Kinshasa,DRC,"(uvira, Kinshasa, DRC)"
9075,vuginyanya,sorogo,uganda,"(vuginyanya, sorogo, uganda)"
9076,wife,Kampala,Uganda,"(wife, Kampala, Uganda)"
9077,wiumiriria,Nyahururu,Kenya,"(wiumiriria, Nyahururu, Kenya)"


### 2.1.d. Merge additional information to the gazetteer

##### Value Counts for Multilocation Tuples

In [13]:
# count the multilocations from the flat list of multilocations
mls_counts = pd.DataFrame( pd.Series( mls_list ).value_counts(), columns=['count_orig'])
                                            
# Diagnostic
print('\nUnique tuples?:')
print(mls_counts.index.is_unique )

print('\nMatching multilocation totals?:')
print('List: \t{}'.format(len(mls_list)))
print('Count: \t{}'.format(mls_counts['count_orig'].sum()))

# Preview
mls_counts


Unique tuples?:
True

Matching multilocation totals?:
List: 	45353
Count: 	45353


,count_orig
"(-1, -1, -1)",2806
"(-1, Nairobi, Kenya)",2449
"(-1, Accra, Ghana)",2053
"(-1, Johannesburg, South Africa)",1757
"(Kawangware, Nairobi, Kenya)",854
...,...
"(Bushbargridge, Bushbargridge, South Africa)",1
"(Rumuruti, Rumuruti, Kenya)",1
"(Dk, Florence, Italy)",1
"(Tillabã©Ri, Tillabã©Ri, Niger)",1


In [14]:
# if it exists, drop the old column - otherwise ignore
G = G.drop(columns=['count_orig'], errors='ignore')

# merge the counts back
G = pd.merge(G, mls_counts,
             left_on = 'tuple_orig',
             right_index = True,
             how='left',
             validate='1:1')

# Diagnostic
print('\nMatching multilocation totals?:')
print('List: \t{}'.format(len(mls_list)))
print('df: \t{}'.format(G['count_orig'].sum()))

# Preview
G


Matching multilocation totals?:
List: 	45353
df: 	45353


,L1_orig,L2_orig,L3_orig,tuple_orig,count_orig
0,,,,"(, , )",1
1,Exhibition,Accra,Ghana,"( Exhibition , Accra, Ghana)",1
2,Kawangware,Nairobi,Kenya,"( Kawangware , Nairobi, Kenya)",1
3,Kayole,Nairobi,Kenya,"( Kayole, Nairobi, Kenya)",3
4,Masimba Junction,Nairobi,Kenya,"( Masimba Junction , Nairobi, Kenya)",1
...,...,...,...,...,...
9074,uvira,Kinshasa,DRC,"(uvira, Kinshasa, DRC)",1
9075,vuginyanya,sorogo,uganda,"(vuginyanya, sorogo, uganda)",1
9076,wife,Kampala,Uganda,"(wife, Kampala, Uganda)",1
9077,wiumiriria,Nyahururu,Kenya,"(wiumiriria, Nyahururu, Kenya)",1


### 2.1.e. Export Gazetteer for Data Cleaning

##### Add additional blank columns to add manual edits in

In [15]:
# add new columns to edit in
G['L1_edit'] = None
G['L2_edit'] = None
G['L3_edit'] = None

# Preview
G

,L1_orig,L2_orig,L3_orig,tuple_orig,count_orig,L1_edit,L2_edit,L3_edit
0,,,,"(, , )",1,None,None,None
1,Exhibition,Accra,Ghana,"( Exhibition , Accra, Ghana)",1,None,None,None
2,Kawangware,Nairobi,Kenya,"( Kawangware , Nairobi, Kenya)",1,None,None,None
3,Kayole,Nairobi,Kenya,"( Kayole, Nairobi, Kenya)",3,None,None,None
4,Masimba Junction,Nairobi,Kenya,"( Masimba Junction , Nairobi, Kenya)",1,None,None,None
...,...,...,...,...,...,...,...,...
9074,uvira,Kinshasa,DRC,"(uvira, Kinshasa, DRC)",1,None,None,None
9075,vuginyanya,sorogo,uganda,"(vuginyanya, sorogo, uganda)",1,None,None,None
9076,wife,Kampala,Uganda,"(wife, Kampala, Uganda)",1,None,None,None
9077,wiumiriria,Nyahururu,Kenya,"(wiumiriria, Nyahururu, Kenya)",1,None,None,None


##### Export to CSV for formatting and upload

In [16]:
G.to_csv('data_gen/gazetteer_multilocations/gazetteer_multilocations_uneditted.csv')